<a href="https://colab.research.google.com/github/r1marcus/Embedded_KI_Workshop/blob/main/Pruning_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing Pruning

In [1]:
!pip install -q tensorflow-model-optimization

In [2]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tempfile
#import tensorflow_model_optimization as tfmot
import numpy as np


# Model configuration
img_width, img_height = 28, 28
batch_size = 250
no_epochs = 10
no_classes = 10
validation_split = 0.2
verbosity = 1

# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()
input_shape = (img_width, img_height, 1)

# Reshape data for ConvNet
input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize [0, 255] into [0, 1]
input_train = input_train / 255
input_test = input_test / 255

# Convert target vectors to categorical targets
target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(input_train, target_train,
          batch_size=batch_size,
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/10
192/192 [==============================] - 34s 176ms/step - loss: 0.7451 - accuracy: 0.7749 - val_loss: 0.1040 - val_accuracy: 0.9722
Epoch 2/10
192/192 [==============================] - 33s 174ms/step - loss: 0.1063 - accuracy: 0.9668 - val_loss: 0.0630 - val_accuracy: 0.9822
Epoch 3/10
192/192 [==============================] - 34s 175ms/step - loss: 0.0702 - accuracy: 0.9785 - val_loss: 0.0474 - val_accuracy: 0.9873
Epoch 4/10
192/192 [==============================] - 34s 175ms/step - loss: 0.0533 - accuracy: 0.9837 - val_loss: 0.0392 - val_accuracy: 0.9890
Epoch 5/10
192/192 [==============================] - 33s 174ms/step - loss: 0.0441 - accuracy: 0.9860 - val_loss: 0.0389 - val_accuracy: 0.9886
Epoch 6/10
192/192 [==============================] - 33s 173ms/step - loss: 0.0365 - accuracy: 0.9884 - val_loss: 0.0436 - val_accuracy: 0.9870
Epoch 7/10
192/192 [==============================] - 33s 174ms/step - loss: 0.0330 - accuracy: 0.9894 - val_loss: 0.0325 - val_ac

# Modell Speichern
Stellen Sie außerdem sicher, dass Sie Ihr Modell in einer temporären Datei speichern, so dass Sie später die Größen des ursprünglichen und des geprunten Modells vergleichen können:

In [4]:

from tensorflow import keras
import tensorflow_model_optimization as tfmot
_, keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model, keras_file, include_optimizer=False)
print(f'Baseline model saved: {keras_file}')

Baseline model saved: /tmp/tmpbz26u9a3.h5


# Loading and configuring pruning

Es lädt die prune_low_magnitude Funktionalität von TensorFlow (Tfmot.sparsity.keras.prune_low_magnitude, n.d.). prune_low_magnitude modifiziert einfach einen Layer, indem es ihn für das Pruning bereit macht. Dies geschieht, indem ein Keras-Modell mit Pruning-Funktionalität umhüllt wird, genauer gesagt, indem sichergestellt wird, dass die Schichten des Modells beschneidbar sind. Damit wird nur die Funktionalität geladen, wir werden sie später tatsächlich aufrufen.

Nach dem Laden des Pruning-Wrappers werden wir die Pruning-Konfiguration festlegen:

In [7]:
# Finish pruning after 5 epochs
pruning_epochs = 5
num_images = input_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * pruning_epochs

# Define pruning configuration
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.40,
                                                               final_sparsity=0.70,
                                                               begin_step=0,
                                                               end_step=end_step)
}
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Hier geschieht Folgendes:

Wir konfigurieren die Länge des Pruning-Prozesses über die Anzahl der Epochen, für die das Modell prunen soll, und nehmen eine Feinabstimmung vor.
Wir laden die Anzahl der Bilder, die in unserem Trainingssatz verwendet werden, abzüglich der Validierungsdaten.
Wir berechnen den end_step unseres Pruning-Prozesses anhand der Stapelgröße, der Anzahl der Bilder sowie der Anzahl der Epochen.
Anschließend definieren wir die Konfiguration für den Pruning-Vorgang über pruning_params. Wir definieren einen Pruning-Zeitplan unter Verwendung von PolynomialDecay, was bedeutet, dass die Sparsamkeit des Modells mit zunehmender Anzahl von Epochen zunimmt. Zu Beginn wird das Modell auf 40 % Sparsamkeit eingestellt und wird zunehmend spärlicher, bis es schließlich 70 % erreicht. Wir beginnen bei 0 und enden bei end_step.
Schließlich rufen wir die Funktion "prune_low_magnitude" (die das beschneidbare Modell erzeugt) mit unserem Ausgangsmodell und den definierten "pruning_params" auf.



# Starten des Beschneidungsvorgangs
Nachdem wir den Pruning-Prozess konfiguriert haben, können wir das Modell neu kompilieren (dies ist notwendig, da wir die Pruning-Funktionalität hinzugefügt haben) und den Pruning-Prozess starten. Wir müssen hier den UpdatePruningStep-Callback verwenden, da er die Optimierungsaktivitäten an den Pruning-Prozess weitergibt (Tfmot.sparsity.keras.UpdatePruningStep, o.J.).

In [8]:
# Recompile the model
model_for_pruning.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Model callbacks
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

# Fitting data
model_for_pruning.fit(input_train, target_train,
                      batch_size=batch_size,
                      epochs=pruning_epochs,
                      verbose=verbosity,
                      callbacks=callbacks,
                      validation_split=validation_split)

Epoch 1/5
192/192 [==============================] - 36s 176ms/step - loss: 0.0219 - accuracy: 0.9926 - val_loss: 0.0332 - val_accuracy: 0.9898
Epoch 2/5
192/192 [==============================] - 34s 176ms/step - loss: 0.0252 - accuracy: 0.9920 - val_loss: 0.0290 - val_accuracy: 0.9915
Epoch 3/5
192/192 [==============================] - 34s 175ms/step - loss: 0.0218 - accuracy: 0.9928 - val_loss: 0.0274 - val_accuracy: 0.9924
Epoch 4/5
192/192 [==============================] - 34s 177ms/step - loss: 0.0173 - accuracy: 0.9945 - val_loss: 0.0277 - val_accuracy: 0.9926
Epoch 5/5
192/192 [==============================] - 34s 177ms/step - loss: 0.0150 - accuracy: 0.9954 - val_loss: 0.0259 - val_accuracy: 0.9929


# Messen der Effektivität des Beschneidens
Sobald das Pruning abgeschlossen ist, müssen wir seine Effektivität messen. Das können wir auf zwei Arten tun:

Indem man misst, wie sehr sich die Leistung im Vergleich zu vor dem Pruning verändert hat;
Indem wir messen, wie sehr sich die Modellgröße im Vergleich zur Zeit vor dem Pruning verändert hat.
Wir tun dies mit den folgenden Codezeilen:

In [9]:
# Generate generalization metrics
score_pruned = model_for_pruning.evaluate(input_test, target_test, verbose=0)
print(f'Pruned CNN - Test loss: {score_pruned[0]} / Test accuracy: {score_pruned[1]}')
print(f'Regular CNN - Test loss: {score[0]} / Test accuracy: {score[1]}')

Pruned CNN - Test loss: 0.022952646017074585 / Test accuracy: 0.9922999739646912
Regular CNN - Test loss: 0.03263925760984421 / Test accuracy: 0.9894999861717224


Diese sind einfach. Sie werten das beschnittene Modell mit den Testdaten aus und geben anschließend das Ergebnis aus, ebenso wie das (zuvor erhaltene) Ergebnis des ursprünglichen Modells.

Als Nächstes exportieren wir es erneut - genau wie zuvor - um sicherzustellen, dass wir es vergleichen können:

In [11]:
# Export the model
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print(f'Pruned model saved: {keras_file}')

Pruned model saved: /tmp/tmpbz26u9a3.h5


Anschließend können wir (dank Pruning Keras Example (n.d.)) die Größe des Keras-Modells vergleichen. Um die Vorteile von Pruning zu verdeutlichen, müssen wir einen Komprimierungsalgorithmus wie gzip verwenden, wonach wir die Größen beider Modelle vergleichen können. Erinnern Sie sich daran, dass Pruning Sparsity erzeugt und dass dünn besetzte Matrizen bei der Komprimierung sehr effizient gespeichert werden können. Deshalb ist gzips für Demonstrationszwecke nützlich. Wir erstellen zunächst eine def, die für die Komprimierung verwendet werden kann, und rufen sie anschließend zweimal auf:

In [12]:
# Measuring the size of your pruned model
# (source: https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras#fine-tune_pre-trained_model_with_pruning)

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))

Size of gzipped baseline Keras model: 1601298.00 bytes
Size of gzipped pruned Keras model: 679955.00 bytes


# Laufzeit-Ergebnis
Nun ist es an der Zeit, es auszuführen. Speichern Sie Ihre Datei als z.B. pruning.py, und führen Sie sie in einer Python-Umgebung aus, in der Sie sowohl tensorflow 2.x als auch numpy und das tensorflow_model_optimization toolkit installiert haben.

Zuerst wird das reguläre Training gestartet, gefolgt vom Pruning-Prozess, und dann wird die Effektivität auf dem Bildschirm angezeigt. Zunächst in Bezug auf die Modellleistung (d. h. Verlust und Genauigkeit):

Pruned CNN - Testverlust: 0.0218335362634185 / Testgenauigkeit: 0.9923999905586243
Regulärer CNN - Testverlust: 0.02442687187876436 / Testgenauigkeit: 0.9915000200271606
Das beschnittene Modell schneidet sogar etwas besser ab als das reguläre Modell. Das liegt wahrscheinlich daran, dass wir das ursprüngliche Modell nur für 10 Epochen trainiert haben und danach mit dem Pruning fortgefahren sind. Es ist sehr gut möglich, dass das Modell noch nicht konvergiert war, sondern sich im Pruning-Prozess weiter in Richtung Konvergenz bewegt hat. Oftmals verschlechtert sich die Performance ein wenig, aber das sollte nur geringfügig sein.

Dann, in Bezug auf die Modellgröße:

Größe des gezippten Keras-Basismodells: 1601609.00 Bytes
Größe des gzipped pruned Keras-Modells: 679958.00 bytes
Code-Sprache: CSS (css)
Das Pruning hat unser Modell definitiv kleiner gemacht - 2,35 Mal!